In [1]:
import arcpy
import numpy as np
import math
#import cv2 as cv
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
from scipy.stats import energy_distance
from scipy.stats import entropy as shannon_entropy
import os
from arcpy.sa import *

In [6]:
def readtensor_1d(filename):
    raster=arcpy.Raster(filename)
    terrain = arcpy.RasterToNumPyArray(raster)
    terrain=terrain.reshape([terrain.size])
    if terrain.min() < 1:
        terrain-=terrain.min()
        terrain+=1
    return terrain

In [8]:
def calShanon(tensor):
    c=np.bincount(tensor)
    c=c[c.nonzero()] #得到类型的histogram
    s = shannon_entropy(c/c.size)
    e = math.log(tensor.size) - (c * np.log(c)).sum() /tensor.size
    return s, e

In [9]:
def calemd_enegery(tensor):
    c=np.bincount(tensor)
    c=c[c.nonzero()] #得到类型的histogram
    types=c.size
    
    #
    grid=np.zeros([types, c.max()], dtype=np.int32)
    for i in range(types):
        grid[i, 0:c[i]] = 1

    h = np.sum(grid, 0).astype(np.float32) #土堆分布

    h1 = np.zeros(tensor.size, np.float32)
    w = np.log(np.arange(tensor.size) + 1) #土堆密度
    
    for i in range(h.size):
        h1[i] = h[i]
    h1 = h1 * w #土堆总量
    
    #h2 = np.ones(h1.size) #土堆分布h1
    
    h2 = np.zeros(h1.size)
    h2[0] = h1.sum()
    
    distance=wasserstein_distance(h1, h2, w, w)
    ##配置熵的计算
    N=np.log(np.arange(tensor.size)+1)
    E=N.sum()-h1.sum()

    distance=energy_distance(h1, h2, w, w)
    dist_norm = 1 - distance/math.sqrt(2)
    return distance, dist_norm,E

In [4]:
def CalEntropy1(reference, method):
    """计算香农熵和配置熵

    Args:
        reference (_type_): _description_
        method (_type_): _description_
    """
    reference_dem = readtensor_1d(reference)
    reference_S = calShanon(reference_dem)
    reference_C = calemd_enegery(reference_dem)
    for dem in method:
        method_dem = readtensor_1d(dem)
        method_S = calShanon(method_dem)
        method_C = calemd_enegery(method_dem)
        diff_S = method_S[0] - reference_S[0]
        diff_C = method_C[2] - reference_C[2] 
        
        print(dem.split("\\")[-1]  + "香农熵：" + str(diff_S))
        print(dem.split("\\")[-1] + "配置熵：" + str(diff_C))

def CalEntropy2(folder, name):
    """批处理计算香农熵和配置熵

    Args:
        folder (_type_): _description_
        name (_type_): _description_
    """
    reference_folder = folder + "\\" + "reference"
    arcpy.env.workspace = reference_folder
    reference_rasters = arcpy.ListRasters()
    for i in range(len(reference_rasters)):
        reference_dem = reference_folder + "\\" + reference_rasters[i]
        method_dems = []
        for n in name:
            method_dem = folder + "\\" + n + "\\" + reference_rasters[i]
            method_dems.append(method_dem)
        CalEntropy1(reference_dem,method_dems)

In [ ]:
folder = r"E:\CCHHYY\超分\大修\添加实验\tfrsa\熵计算"
name = ["TfaSR"]
CalEntropy2(folder, name)